In [1]:
# -*- coding: utf-8 -*-
import tensorflow as tf
from gensim.models.word2vec import Word2Vec
from gensim.corpora.dictionary import Dictionary
import numpy as np
import sys
import pickle
import os
# import from keras.preprocessing import sequence
import re

In [2]:
datasource_dir = r'/home/alexsun/ML/data_center/standford_large_movie_review_dataset/aclImdb/'

class Mysentences(object):
    def __init__(self, dir_name):
        self.dir_name = dir_name
        
    @staticmethod
    def multiple_replace(text, adict):
        rx = re.compile('|'.join(map(re.escape, adict)))
        def one_xlat(match):
            return adict[match.group(0)]
        return rx.sub(one_xlat, text)
    
    @staticmethod
    def txt_process(txt):
        text_replace_dic = {'<br />':'', '\n':' ', '.':'', '?':'', '*':'', '$':''}
        return Mysentences.multiple_replace(txt, text_replace_dic)
    
    def __iter__(self):
        for filename in os.listdir(self.dir_name):
            with open(os.path.join(self.dir_name, filename), encoding='utf-8') as f:
                new_line = Mysentences.txt_process(f.read())
                yield new_line.split()
                
# print([line for line in Mysentences('/home/alexsun/aha')])
               

In [3]:
# 利用word2vec训练出来的模型，　返回每个词语的索引，　词向量
# 参考：https://github.com/BUPTLdy/Sentiment-Analysis/blob/master/code/Sentiment_lstm.py
def create_dics(model):
    if (model is not None):
        gensim_dict = Dictionary()
        gensim_dict.doc2bow(model.wv.vocab.keys(),
                          allow_update=True)
        w2index = {v:k+1 for k,v in gensim_dict.items()} # 仅仅保留出现次数大于min_count的单词索引
        w2vec = model.wv
        # 保留这句话，　可以在投喂神经网络计算前加上     
        # combined_vec = sequence.pad_sequences(combined, maxlen=maxlen)               
    else:
        print('No data provided')
    return w2index, w2vec   

pickle_path = os.path.join(datasource_dir, 'pickles/')

def word2vec_train():
    print('w2v method begins')
    n_iterations = 4
    total_examples = 50002
    save_pickle_word2vec = pickle_path + 'word2vec.pickle'
    if(os.path.exists(save_pickle_word2vec)):
        print('word2vec training process ignored, file exists... ')
        model = Word2Vec.load(save_pickle_word2vec)
    else:
        sentences = Mysentences(os.path.join(datasource_dir, 'total/'))
        model = Word2Vec(size=100, window=5, min_count=10, workers=4)
        model.build_vocab(sentences)
        # model.sort_vocab()
        print('vocab builded')
        # i dont know what's epochs, but it must be passed
        model.train(sentences, total_examples=total_examples, epochs=10)
        print('model trained!')
        model.save(save_pickle_word2vec)
        print('word2vec model has been trained! and the model has been saved')
    index2word, word_vectors = create_dics(model=model)
    return model, index2word, word_vectors
    # return model


model, word2index, word_vectors = word2vec_train()
print('word2idex: the ', word2index['the'])
print('total number of words: %d' %(len(word2index.keys())))

w2v method begins
word2vec training process ignored, file exists... 
word2idex: the  642
total number of words: 36763


In [4]:
# simple test block
# print(word_vectors['funny'])
# print(word_vectors['interesting'])
print(model.similarity('bus', 'fun'))
print(model.similarity('bus', 'car'))
print(model.similarity('China', 'USA'))
print(model.similarity('China', 'fun'))
print(model.similarity('Queen', 'King'))
print(model.similarity('Queen','car'))
print(model.similarity('Korea', 'USA'))

-0.0174416840107
0.626315284298
0.665314994582
-0.0225523624497
0.550402141467
0.082536765306
0.570134050151
